In [1]:
batch_size_for_classification = 8000
#prediction
prediction_path = "Export/cdnow/cdnow_prediction_trmf_variantA_"+str(batch_size_for_classification+2000)+'_.csv'

In [2]:
import numpy as np
import pandas as pd
import trmf
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from trmf import TRMFRegressor
import pandas as pd
import datetime
import random
import rpy2
import math
from sklearn.model_selection import KFold, GridSearchCV
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Path fo import files
#input data
import_data = "import/CDNOW_master.txt"
# clusters
import_clusters = "clusters/results_cdnow.csv"

### 0. Preprocessing cdnow

In [4]:
df = pd.read_csv(import_data,header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
12826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.1 Choose TS with given clusters and divide into test/train dataset

In [5]:
cluster_TS  = pd.read_csv(import_clusters)
cluster_TS.head()

,Unnamed: 0,clusters_r,clusters_m,clusters_f
0,11770,0,0,0
1,14675,17,0,8
2,10026,2,0,3
3,13997,1,0,2
4,741,0,0,0


In [6]:
df_train = cluster_TS.merge(df,left_on = "Unnamed: 0", right_on = 'id').drop(columns = ['clusters_r','clusters_m','clusters_f'])
df_train = df_train.rename(index = str, columns={"Unnamed: 0": "id"}).set_index('id')
df_train.head()

,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,43.13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_test= df.drop(df_train.index.values,axis=0).sample(batch_size_for_classification)
df_test.head()

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
17103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### NAN -> 0

In [8]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train.sample(2)

,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
14129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Split into train/test (last 70 days) and scaling

In [9]:
train_targets, test_targets= train_test_split((df_test.append(df_train)).transpose(), test_size=.128, shuffle=False)

In [10]:
test_targets.shape

(70, 10000)

In [11]:
test_targets = test_targets.transpose()
train_targets = train_targets#.transpose()

### TRMF

### GridSearch

In [12]:
grid = ParameterGrid(dict(
    n_components=np.r_[1:2],
    n_order=np.r_[1:2],
    C_Z=np.logspace(-2, 1, num=1),
    C_F=np.logspace(-2, 1, num=1),
    C_phi=np.logspace(-2, 1,num=1),
    eta_Z=np.linspace(0.05, 0.95,num=2),
))

In [13]:
base = TRMFRegressor(n_components=1,
                     n_order=0,
                     fit_regression=False,
                     fit_intercept=True,
                     nonnegative_factors=True,
                     n_max_mf_iter=5)


transformer = StandardScaler(with_mean=True, with_std=True)

In [14]:
def helper(par, train, test, base=base,
           transformer=transformer):
    # clone, set parameters and fit
    trmf = clone(base).set_params(**par).fit(train)
    
    # predict and return
    pred = transformer.inverse_transform(
        trmf.predict(n_ahead=len(test)))

    return par, mean_squared_error(test, pred)

X = transformer.fit_transform(train_targets)

In [15]:
%%time
# run the experiment in parallel
par_ = Parallel(n_jobs=-1, verbose=0)
results = par_(delayed(helper)(par, X, test_targets.transpose()) for par in grid)

CPU times: user 68 ms, sys: 524 ms, total: 592 ms
Wall time: 1min 44s


In [16]:
keys = ['n_order', 'n_components', 'eta_Z', 'C_phi', 'C_Z', 'C_F']
data = dict((tuple(par[k] for k in keys), rmse,) for par, rmse in results)
sr = pd.Series(data, name="rmse").sort_index().rename_axis(keys)
cube = sr.values.reshape(*[len(grid.param_grid[0][k]) for k in keys])
stepping = [grid.param_grid[0][k] for k in keys]
# find the flat index of the smallest value
flat_index = np.argmin(cube)
# ... and unravel into into a multidimensional index
index = np.unravel_index(flat_index, cube.shape)
# collect the best paramaters from the grid
best_ = {k: grid.param_grid[0][k][i] for k, i in zip(keys, index)}

In [17]:
best_

{'n_order': 1,
 'n_components': 1,
 'eta_Z': 0.05,
 'C_phi': 0.01,
 'C_Z': 0.01,
 'C_F': 0.01}

### Prediction

In [18]:
#scaling
scl = StandardScaler(with_mean=True, with_std=True)
train_targets_scaled = scl.fit_transform(train_targets)

In [19]:
%%time
par = {
    'n_order': 7,
    'n_components': 12,
    'eta_Z': 0.95,
    'C_phi': 0.1,
    'C_Z': 10.0,
    'C_F': 0.01,
}
trmf = TRMFRegressor(**best_, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
trmf.fit(train_targets_scaled)

CPU times: user 1min 26s, sys: 8min 32s, total: 9min 58s
Wall time: 1min 5s


In [20]:
# We predict 70 days
n_ahead = test_targets.shape[1]

In [21]:
#predicted = trmf.predict(X=(train_targets_scaled), n_ahead=n_ahead)
predicted = scl.inverse_transform(trmf.predict(X=(train_targets_scaled), n_ahead=n_ahead))

In [22]:
df_prediction = pd.DataFrame(predicted).transpose()
#df_prediction = pd.DataFrame(scl.transform(predicted)).transpose()

In [23]:
print("RMSE score:", round(math.sqrt(mean_squared_error(test_targets,df_prediction)),2))

RMSE score: 3.14


### Transform prediction for output

In [24]:
df_prediction.columns  = test_targets.columns 
df_prediction.index = test_targets.index
#MODEL
df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(test_targets,df_prediction)))
df_prediction.insert(0,'model','trmf')
#METHOD
df_prediction.insert(0,'method','all_data')
#STEP
df_prediction.insert(0,'step','all_data')
# CLUSTER
df_prediction.insert(0,'cluster_id','none')
#Wide to logn
df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
true_values=test_targets.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
df_prediction = df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date'])

### Save to file 

In [25]:
df_prediction.to_csv(prediction_path)
df_prediction.sample(5)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
66651,17210,none,all_data,all_data,trmf,3.142905,1998-04-28,0.110105,0.0
339081,17138,none,all_data,all_data,trmf,3.142905,1998-05-25,0.010063,0.0
85127,10965,none,all_data,all_data,trmf,3.142905,1998-04-30,0.210525,0.0
510436,936,none,all_data,all_data,trmf,3.142905,1998-06-12,0.031492,0.0
388750,13904,none,all_data,all_data,trmf,3.142905,1998-05-30,0.071891,0.0


In [26]:
# To get full analysis, please run "combine_predictions.ipynb" file, which combines the prediction files of all approaches